<a href="https://colab.research.google.com/github/053steve/CRNN-model-text-recognition/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup all basic stuff for colab and github integration

In [1]:

COLAB = False
GIT_REPOSITORY = "CRNN-model-text-recognition"
FILE_NAME = "notebook.ipynb"

if COLAB:    
    # PARENT_DIRECTORY_PATH = "/content"
    # In case you want to clone in your drive:
    PARENT_DIRECTORY_PATH = "/content/gdrive/My Drive/Projects"
    DATASET_DIRECTORY_PATH = "/content/gdrive/My Drive/datasets/captcha"
    SOURCE_DATASET_ZIP_FILE = "archive.zip"
    SOURCE_DATASET_ZIP_PATH = DATASET_DIRECTORY_PATH + "/" + SOURCE_DATASET_ZIP_FILE
    PROJECT_PATH = PARENT_DIRECTORY_PATH + "/" + GIT_REPOSITORY
    %cd "{PARENT_DIRECTORY_PATH}"
    from google.colab import drive
    drive.mount('/content/gdrive')
else:
    DATASET_DIRECTORY_PATH = "/content/gdrive/My Drive/datasets/captcha"
    SOURCE_DATASET_ZIP_FILE = "archive.zip"
    SOURCE_DATASET_ZIP_PATH = DATASET_DIRECTORY_PATH + "/" + SOURCE_DATASET_ZIP_FILE

/content/gdrive/My Drive/Projects
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18

import string
from tqdm.notebook import tqdm
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import multiprocessing as mp


In [9]:
cpu_count = mp.cpu_count()
print(cpu_count)

8


In [ ]:
print("unzip source in path " + SOURCE_DATASET_ZIP_PATH)
!unzip '/content/gdrive/My Drive/datasets/captcha/archive.zip'
